# How to get TLE data from [Space-Track.org](https://www.space-track.org/auth/login)

### Using Selenium

Before you start make sure that you have ChromeDriver on your computer 
if in case you do not download it from this [link](https://sites.google.com/a/chromium.org/chromedriver/home). <br>
Then add this chromedriver.exe to your PATH.

In [1]:
# Import necessary modules 

from selenium import webdriver
import numpy as np
import pandas as pd
import re 

In [13]:
# Create a chrome driver that automates the chrome webbrowser operations

driver = webdriver.Chrome()

# Open Space-Track.org login page with the url 
space_track_url = 'https://www.space-track.org/auth/login'
driver.get(space_track_url)

# Assert for any errors concerning the url of the login page 
assert 'space' or 'track' in driver.title, 'Improper url for Space-Track.org'

In [13]:
# Enter the users Space-Track.org username and password manually 

un = input('Enter your username -> ')
pw = input('Enter your password -> ')

In [13]:
# Enter username
username = driver.find_element_by_id('identity')
username.clear()
username.send_keys(un)

# Enter password 
password = driver.find_element_by_id('password')
password.clear()
password.send_keys(pw)

# Click the login button 
driver.find_element_by_name('btnLogin').click()

# Click the tab for Recent TLEs 

recent_TLE_url = '#recent'
driver.find_element_by_xpath('//a[@href="'+recent_TLE_url+'"]').click()

# Openning the TLe data page 

TLE_data_url = "https://www.space-track.org/basicspacedata/query/class/tle_latest/ORDINAL/1/EPOCH/%3Enow-30/orderby/NORAD_CAT_ID/format/tle"
driver.find_element_by_xpath('//a[@href="'+TLE_data_url+'"]').click()

# Switch driver to the new tab

driver.switch_to.window(driver.window_handles[1])

In [14]:
# Check that the driver is on the right tab

driver.current_url

'https://www.space-track.org/basicspacedata/query/class/tle_latest/ORDINAL/1/EPOCH/%3Enow-30/orderby/NORAD_CAT_ID/format/tle'

In [70]:
# Get the data

data_text = driver.find_element_by_xpath("//body/pre").text

# Split the text by newlines 

text_by_line = data_text.splitlines()

### Using Requests

#### Changes made: We will be using the space-track.org query API to retrieve the data once we login to the website

In [7]:
import requests
from bs4 import BeautifulSoup as bs

In [8]:
# What is the login page

url = 'https://www.space-track.org/'
login_route = 'auth/login'

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
}


In [3]:
# Enter the users Space-Track.org username and password manually 

un = input('Enter your username -> ')
pw = input('Enter your password -> ')

Enter your username ->  smallpond.eng.tom@gmail.com
Enter your password ->  svG*kfr6LB5zUyL


In [9]:
# Get the HTML that we want 

with requests.session() as client:
    # Get csrftoken
    cookiejar = client.get(url + login_route).cookies
    cookies = cookiejar.get_dict(domain='www.space-track.org') 
    cookie = cookies['spacetrack_csrf_cookie']  
    
    # What you have to enter to login 
    login_payload = {
        'spacetrack_csrf_token': cookie,
        'identity': un,
        'password': pw,
        'btnLogin': 'LOGIN'
    }
    
    # Post the data to login 
    login_request = client.post(url + login_route, headers=HEADERS, data=login_payload)
    
    # Check if we are logged in or not 
    assert login_request.status_code == 200, 'Not logged in.'  
    cookies = login_request.cookies  # save the cookies for the login 
    
    # Use beautiful soup to extract the HTML that we want 
    api_query = 'https://www.space-track.org/basicspacedata/query/class/gp/EPOCH/%3Enow-30/orderby/NORAD_CAT_ID,EPOCH/format/3le'
    TLE_data_page = client.get(api_query) 
    soup = bs(TLE_data_page.text, 'html.parser')

    TLE_data_1string = str(soup.text)

In [10]:
# Split the long string by newlines and make it an array of strings 

TLE_data = TLE_data_1string.splitlines()

In [11]:
TLE_data

['0 VANGUARD 1',
 '1     5U 58002B   20270.72609959 +.00000008 +00000-0 +35143-4 0  9996',
 '2     5 034.2447 276.2975 1845776 088.0693 292.9082 10.84868479216254',
 '0 VANGUARD 2',
 '1    11U 59001A   20270.83057634 -.00000014 +00000-0 -52792-5 0  9991',
 '2    11 032.8687 034.7609 1467087 103.7620 273.0083 11.85679410287012',
 '0 VANGUARD R/B',
 '1    12U 59001B   20270.89554473 +.00000372 +00000-0 +19820-3 0  9999',
 '2    12 032.9137 356.0609 1666130 249.2921 092.1676 11.44320987291609',
 '0 VANGUARD R/B',
 '1    16U 58002A   20270.85942587 -.00000022 +00000-0 -46274-4 0  9994',
 '2    16 034.2702 221.8316 2022572 305.2573 037.4791 10.48675665469485',
 '0 VANGUARD 3',
 '1    20U 59007A   20270.48712837 +.00000328 +00000-0 +14121-3 0  9999',
 '2    20 033.3411 182.1436 1666591 082.6853 295.9308 11.55733657242298',
 '0 EXPLORER 7',
 '1    22U 59009A   20270.68850821 +.00000323 +00000-0 +46901-4 0  9998',
 '2    22 050.2861 347.0443 0139436 285.4847 073.0739 14.95205750485528',
 '0 TI